In [1]:
import basedosdados as bd
import pandas as pd
import mysql.connector
import numpy as np
import re
import pycep_correios
import matplotlib.pyplot as plt
import seaborn as sns
import time
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', None)
pd.reset_option('^display.', silent=True)

In [ ]:
#salva dataframe em csv
#df.to_csv('combustiveis.csv', index=False, sep=';', encoding='utf-8')

In [65]:
#lendo csv
df = pd.read_csv('combustiveis.csv', sep=';')
#csv_2021_1
df_2021 = pd.read_csv('https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2021-01.csv', sep=';', encoding='latin-1')
#csv_2021_2
df_2021_2 = pd.read_csv('https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2021-02.csv', sep=';', encoding='latin-1')
#csv_2022_1
df_2022_1 = pd.read_csv('ca-2022-01.csv', sep=';', encoding='latin-1')


In [3]:
#ver o tipo das colunas
df.dtypes

ano                       int64
sigla_uf                 object
id_municipio              int64
bairro_revenda           object
cep_revenda              object
endereco_revenda         object
cnpj_revenda             object
nome_estabelecimento     object
bandeira_revenda         object
data_coleta              object
produto                  object
unidade_medida           object
preco_compra            float64
preco_venda             float64
dtype: object

In [4]:
df

,ano,sigla_uf,id_municipio,bairro_revenda,cep_revenda,endereco_revenda,cnpj_revenda,nome_estabelecimento,bandeira_revenda,data_coleta,produto,unidade_medida,preco_compra,preco_venda
0,2012,PA,1501402,MARAMBAIA,66615-000,"AVENIDA TAVARES BASTOS,654",00.590.313/0001-47,J CARLOS SILVA COMERCIO EIRELI - EPP,BRANCA,2012-03-07,diesel s50,R$/litro,1.9219,2.199
1,2021,MG,3136702,INDUSTRIAL,36081-000,"AVENIDA PRESIDENTE JUSCELINO KUBITSCHECK,130",22.169.455/0001-78,POSTO FACIT LTDA,RAIZEN,2021-05-10,gasolina aditivada,R$/litro,NaN,6.059
2,2021,BA,2927408,PITUACU,41740-090,"AVENIDA PROFESSOR PINTO DE AGUIAR,650",04.420.464/0003-70,ECOPOSTOS - COMBUSTIVEIS E SERVICOS LTDA,PETROBRAS DISTRIBUIDORA S.A.,2021-05-10,gasolina aditivada,R$/litro,NaN,5.999
3,2021,RN,2408102,CANDELARIA,59065-780,"AVENIDA GOV. TARCISIO DE VASCONCELOS MAIA,1450 A",12.689.295/0002-15,FLOR E OLIVEIRA LTDA,PETROBRAS DISTRIBUIDORA S.A.,2021-05-11,gasolina aditivada,R$/litro,NaN,5.950
4,2021,SP,3556206,VILA OLIVO,13276-020,"RUA TREZE DE MAIO,414",46.088.712/0001-05,COMERCIO DE LUBRIFICANTES VALESSO LTDA,BRANCA,2021-05-12,gasolina aditivada,R$/litro,NaN,5.259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26514819,2013,MG,3131703,CENTRO,35900-000,"AVENIDA JOAO PINHEIRO,665",20.963.146/0001-40,COMERCIO E TRANSPORTES RODRIGUES LTDA,IPIRANGA,2013-12-30,diesel s10,R$/litro,NaN,2.690
26514820,2013,RJ,3304201,PARAÍSO,27534-010,"RODOVIA PRESIDENTE DUTRA,S/N KM 305",29.073.905/0001-73,PARAISO LUBRIFICANTES LTDA,RAIZEN,2013-12-30,diesel s10,R$/litro,NaN,2.490
26514821,2013,BA,2906501,DISTRITO INDUSTRIAL,43813-300,"RODOVIA BA 522 KM 10,SN",06.099.586/0001-70,POSTO DE COMBUSTIVEIS E LUBRIFICANTES CORREIA ...,BRANCA,2013-12-31,diesel s10,R$/litro,2.3021,2.630
26514822,2013,ES,3201308,CAMPO GRANDE,29146-341,"ROD. BR 262,S/N KM 03",05.964.723/0001-24,LUBE COMÉRCIO DE PETRÓLEO LTDA.,IPIRANGA,2013-12-31,diesel s10,R$/litro,2.2639,2.730


In [5]:
#tranforma coluna de data em date
df['data_coleta'] = pd.to_datetime(df['data_coleta'])

In [6]:
df['produto'].unique()

array(['diesel s50', 'gasolina aditivada', 'etano', 'glp', 'gnv',
       'diesel', 'etanol', 'gasolina', 'diesel s10'], dtype=object)

In [26]:
#criar dataframe com valor maximo e minimo de cada produto com valor de compra
df_max_compra = df.groupby('produto')['preco_compra'].max().reset_index()
df_min_compra = df.groupby('produto')['preco_compra'].min().reset_index()
#juntar os dois dataframes
df_max_min_compra = pd.merge(df_max, df_min, on='produto')
#renomear as colunas
df_max_min_compra.columns = ['produto', 'preco_compra_max', 'preco_compra_min']
#verificar o dataframe
df_max_min_compra


,produto,preco_compra_max,preco_compra_min
0,diesel,4.2700,0.73067
1,diesel s10,4.4782,1.71000
2,diesel s50,2.4670,1.29850
3,etanol,4.6929,0.32570
4,gasolina,5.0910,1.11110
5,gasolina aditivada,NaN,NaN
6,glp,94.5000,13.54000
7,gnv,4.2189,0.36320


In [27]:
#criar dataframe com valor maximo e minimo de cada produto com valor de venda
df_max_venda = df.groupby('produto')['preco_venda'].max().reset_index()
df_min_venda = df.groupby('produto')['preco_venda'].min().reset_index()
#juntar os dois dataframes
df_max_min_venda = pd.merge(df_max_venda, df_min_venda, on='produto')
#renomear as colunas
df_max_min_venda.columns = ['produto', 'preco_venda_max', 'preco_venda_min']
#verificar o dataframe
df_max_min_venda

,produto,preco_venda_max,preco_venda_min
0,diesel,6.480,1.020
1,diesel s10,6.799,1.938
2,diesel s50,2.950,1.599
3,etanol,7.199,0.590
4,gasolina,7.499,1.390
5,gasolina aditivada,7.987,3.457
6,glp,135.000,21.990
7,gnv,5.949,0.659


In [28]:
#removendo linha com valor do produto etano
df = df[df['produto'] != 'etano']
#colocar Nao informado no valores nulos da coluna bairro revenda
df['bairro_revenda'].fillna('Nao informado', inplace=True)
#removendo gasolina aditivada do dataframe
df = df[df['produto'] != 'gasolina aditivada']

In [29]:
#caculando a diferença do preço de compra e venda
df['diferenca'] = df['preco_venda'] - df['preco_compra']

In [31]:
#criar um dataframe com a moda da diferença por tipo de produto
df_moda = df.groupby('produto')['diferenca'].agg(pd.Series.mode).reset_index()
df_moda

,produto,diferenca
0,diesel,0.2100
1,diesel s10,0.3100
2,diesel s50,0.2300
3,etanol,0.2500
4,gasolina,0.4000
5,glp,7.0000
6,gnv,0.2667


In [32]:
#preencher os valores nulos da preco_compra com a moda da diferença por tipo de produto
df['preco_compra'] = df['preco_compra'].fillna(df['produto'].map(df_moda.set_index('produto')['diferenca']))

In [33]:
#somar os valores nulos da coluna preco_compra por produto
df['preco_compra'].isnull().groupby(df['produto']).sum()

produto
diesel        0
diesel s10    0
diesel s50    0
etanol        0
gasolina      0
glp           0
gnv           0
Name: preco_compra, dtype: int64

In [34]:
#verificar se existe algum valor nulo
df.isnull().sum()

ano                            0
sigla_uf                       0
id_municipio                   0
bairro_revenda                 0
cep_revenda                    0
endereco_revenda               0
cnpj_revenda                   0
nome_estabelecimento           0
bandeira_revenda               0
data_coleta                    0
produto                        0
unidade_medida                 0
preco_compra                   0
preco_venda                    0
diferenca               14290145
dtype: int64

In [35]:
#remover a coluna data['diferenca']
df.drop('diferenca', axis=1, inplace=True)

In [38]:
#criar data frames conforme o tipo de combustível
#gasolina s50
df_diesel_s50 = df[df['produto'] == 'diesel s50']
#etanol
df_etanol = df[df['produto'] == 'etanol']
#glp (gás de cozinha)
df_glp = df[df['produto'] == 'glp']
#gnv (gás natural veicular)
df_gnv = df[df['produto'] == 'gnv']
#diesel
df_diesel = df[df['produto'] == 'diesel']
#gasolina
df_gasolina_comum = df[df['produto'] == 'gasolina']
#diesel s10
df_diesel_s10 = df[df['produto'] == 'diesel s10']


In [39]:
#criar lista com os data frames
lista_nomes = ['df_gasolina_comum', 'df_etanol', 'df_diesel', 'df_gnv', 'df_glp', 'df_diesel_s10', 'df_diesel_s50']
lista_df = [df_gasolina_comum, df_etanol, df_diesel, df_gnv, df_glp, df_diesel_s10, df_diesel_s50]

In [40]:
#ver somente as colunas com valores nulos de cada dataframem
#trazer somente as colunas com valores nulos de cada dataframem e quantos valores nulos tem em cada coluna
def mostra_colunas_nulas(lista_df, lista_nomes):
    for i in range(len(lista_df)):
        print(f'{lista_nomes[i]} tem as seguintes colunas com valores nulos: ', end=' ')
        print(lista_df[i].columns[lista_df[i].isnull().any()].tolist(), end=' ')
        print('e a quantidade de valores nulos em cada coluna é: ', end=' ')
        #não trazer os valores zerados
        print(lista_df[i].isnull().sum()[lista_df[i].isnull().sum() != 0].tolist())
        print('')

In [41]:
#funação para ver a quantidade de linhas e colunas de cada dataframe
def mostra_dimensoes(lista_df, lista_nomes):
    for i in range(len(lista_df)):
        print(f'{lista_nomes[i]} tem um total de', end=' ')
        print('{} linhas e {} colunas'.format(lista_df[i].shape[0], lista_df[i].shape[1]))
        print('')

In [42]:

mostra_colunas_nulas(lista_df, lista_nomes)

df_gasolina_comum tem as seguintes colunas com valores nulos:  [] e a quantidade de valores nulos em cada coluna é:  []

df_etanol tem as seguintes colunas com valores nulos:  [] e a quantidade de valores nulos em cada coluna é:  []

df_diesel tem as seguintes colunas com valores nulos:  [] e a quantidade de valores nulos em cada coluna é:  []

df_gnv tem as seguintes colunas com valores nulos:  [] e a quantidade de valores nulos em cada coluna é:  []

df_glp tem as seguintes colunas com valores nulos:  [] e a quantidade de valores nulos em cada coluna é:  []

df_diesel_s10 tem as seguintes colunas com valores nulos:  [] e a quantidade de valores nulos em cada coluna é:  []

df_diesel_s50 tem as seguintes colunas com valores nulos:  [] e a quantidade de valores nulos em cada coluna é:  []



In [43]:
mostra_dimensoes(lista_df, lista_nomes)

df_gasolina_comum tem um total de 7066140 linhas e 14 colunas

df_etanol tem um total de 6554200 linhas e 14 colunas

df_diesel tem um total de 5231119 linhas e 14 colunas

df_gnv tem um total de 405778 linhas e 14 colunas

df_glp tem um total de 5413252 linhas e 14 colunas

df_diesel_s10 tem um total de 1670878 linhas e 14 colunas

df_diesel_s50 tem um total de 44495 linhas e 14 colunas

